In [1]:
require 'nn'
require 'hdf5'

local Squeeze, parent = torch.class('nn.Squeeze', 'nn.Module')

function Squeeze:updateOutput(input)
    self.size = input:size()
    self.output = input:squeeze()
    return self.output
end

function Squeeze:updateGradInput(input, gradOutput)
  self.gradInput = gradOutput:view(self.size)
  return self.gradInput  
end

In [2]:
myFile = hdf5.open('4-grams.hdf5','r')
data = myFile:all()
myFile:close()

In [ ]:
neuralnet = nn.Sequential()
neuralnet:add(nn.LookupTable(data['nwords'][1],dim_hidden))
neuralnet:add(nn.View(1,-1,5*50))
neuralnet:add(nn.Squeeze()) -- this layer is to go from a 1xAxB tensor to AxB dimensional tensor (https://groups.google.com/forum/#!topic/torch7/u4OEc0GB74k)
neuralnet:add(nn.Linear(5*dim_hidden,dim_hidden2))
neuralnet:add(nn.HardTanh())
neuralnet:add(nn.Linear(dim_hidden2, data['nclasses'][1]))
neuralnet:add(nn.LogSoftMax())

In [ ]:
criterion = nn.ClassNLLCriterion()

In [ ]:
dataset={};
for i=1,train:size(1) do 
  dataset[i] = {train[i]:view(1,5), train_output[i]}
end
function dataset:size() return train:size(1) end -- 100 examples

In [ ]:
trainer = nn.StochasticGradient(neuralnet, criterion)
trainer.learningRate = 0.01
trainer.maxIteration = 10
trainer:train(dataset)